# 1. 파일 만들기

In [ ]:
import csv
import os

file_path = "../data/hand_data.csv"

if not os.path.exists(file_path):
    # 위 경로에 파일이 없을 때 파일 생성 후 데이터 추가
    with open(file_path, "w") as file:
        writer = csv.writer(file)
        writer.writerow([1,2,3,4,5])
else:
    # 파일에 데이터 추가하기
    with open(file_path, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([1,2,3,4,5])

In [ ]:
# hand landmark 옵션
# (New) 저장할 파일명 넣기 file_path
# (New) 만약에 파일경로가 없으면 새로 만들기
# 웹캠
    # 카메라 감지
    # 좌우 반전
    # 손 그리기 준비
    # 손 감지
    # Hand Landmark 추출
        # 손 하나의 Hand Landmark 추출
        # 저장 데이터 만들기, 포인트 그리기
        # 1누르면 rock, 2누르면 scissors, 3누르면 paper
    # 화면 띄우기
    # ESC 누르면 종료
# 마무리

# 2. 데이터 학습

## 1) 데이터 불러오기

In [55]:
#  pandas 필요 > uv add pandas
import pandas as pd

df = pd.read_csv("C:\Potenup\OpenCV-Study\data\hand_data.csv", header=None)

In [56]:
len(df.columns)

64

In [57]:
df.rename(columns = {63 : 'label'}, inplace=True)

In [58]:
df.head()

0         1             2         3         4         5         6  \
0  0.499530  0.677184 -6.194429e-07  0.443475  0.638663  0.000904  0.407683   
1  0.501756  0.670150 -5.939269e-07  0.451522  0.626631 -0.000168  0.421733   
2  0.521650  0.663296 -5.203168e-07  0.465848  0.614003 -0.000228  0.439991   
3  0.552225  0.651288 -4.625390e-07  0.543388  0.566807  0.017640  0.513363   
4  0.556297  0.663771 -4.211632e-07  0.556154  0.573791  0.023446  0.533182   

          7         8         9  ...        54        55        56        57  \
0  0.584179 -0.002717  0.395593  ...  0.481943  0.494438 -0.042373  0.485666   
1  0.573628 -0.007440  0.406904  ...  0.475028  0.495256 -0.048250  0.479898   
2  0.559969 -0.007519  0.420879  ...  0.455851  0.505166 -0.044188  0.463470   
3  0.509916  0.013313  0.463296  ...  0.428735  0.534691 -0.053328  0.435399   
4  0.510428  0.023017  0.491027  ...  0.425548  0.543261 -0.041568  0.437400   

         58        59        60        61        62  label  
0  0.540222 -0.031663  0.505677  0.551543 -0.017468   rock  
1  0.540075 -0.039471  0.503290  0.548613 -0.026694   rock  
2  0.546609 -0.038495  0.488143  0.555869 -0.029265   rock  
3  0.567599 -0.051424  0.450033  0.578540 -0.050367   rock  
4  0.574760 -0.035259  0.454282  0.582239 -0.032377   rock  

[5 rows x 64 columns]

## 2) 데이터 분할하기

In [61]:
df['label'].unique()

array(['rock', 'Sissors', 'Paper'], dtype=object)

In [64]:
# sklean 필요 > uv add scikit-learn
from sklearn.preprocessing import LabelEncoder

label_en = LabelEncoder()

X = df.drop('label', axis=1)
y = label_en.fit_transform(df['label'])

In [65]:
label_en.classes_

array(['Paper', 'Sissors', 'rock'], dtype=object)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## 2) 모델 만들기

In [47]:
from sklearn.metrics import accuracy_score, classification_report

def train_model(model, X_train, y_train):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

def evaluate_model(y_test, y_pred):
    print("정확도:", accuracy_score(y_test, y_pred))
    print(classification_report(y_pred, y_test))

In [51]:
# xgboost 필요 > uv pip install xgboost
from xgboost.sklearn import XGBClassifier

model = XGBClassifier(
            n_estimators=50, ## 붓스트랩 샘플 개수 또는 base_estimator 개수
            max_depth=5, ## 개별 나무의 최대 깊이
            gamma = 0, ## gamma
            importance_type='gain', ## gain, weight, cover, total_gain, total_cover
            reg_lambda = 1, ## tuning parameter of l2 penalty
            random_state=100
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=0, grow_policy=None,
              importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, ...)

## 4) 모델 평가하기

In [53]:
y_pred = model.predict(X_test)
evaluate_model(y_test, y_pred)

정확도: 0.9777777777777777
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        15
           1       0.93      1.00      0.97        14
           2       1.00      0.94      0.97        16

    accuracy                           0.98        45
   macro avg       0.98      0.98      0.98        45
weighted avg       0.98      0.98      0.98        45



## 5) 모델 저장하기

In [54]:
import joblib

# 모델 저장
joblib.dump(model, '../models/rock-sission-papaer-model.pkl')

['../models/rock-sission-papaer-model.pkl']